In [1]:
import pandas as pd

# Load recipe data from CSV
recipes_df = pd.read_csv('../Resource/completed_recipes.csv')
print(f"Loaded {len(recipes_df)} recipes.")


Loaded 522517 recipes.


In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "D*d4-0+Kl+lxfbbzh5ut"),
    ca_certs="~/http_ca.crt"
)

if es.ping():
    print("Elasticsearch connect for now!!")
else:
    print("failed to connect")

Elasticsearch connect for now!!


In [ ]:
import json
import numpy as np
from elasticsearch.helpers import bulk
import re

# Define index name and sample size for development
index_name = "recipes"
sample_size = 1000  # Set the sample size for testing (adjust as needed)

# Delete the index if it already exists
es.indices.delete(index=index_name, ignore=[400, 404])

mapping = {
    "settings": {
        "analysis": {
            "tokenizer": {
                "ngram_tokenizer": {
                    "type": "ngram",
                    "min_gram": 2,
                    "max_gram": 3,
                    "token_chars": ["letter", "digit"]
                }
            },
            "filter": {
                "shingle_filter": {
                    "type": "shingle",
                    "min_shingle_size": 2,
                    "max_shingle_size": 3
                }
            },
            "analyzer": {
                "default": {
                    "type": "english"
                },
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "ngram_tokenizer",
                    "filter": ["lowercase"]
                },
                "shingle_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": ["lowercase", "shingle_filter"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "recipe_id": {"type": "keyword"},
            "name": {
                "type": "text",
                "analyzer": "english",
                "fields": {
                    "ngram": {
                        "type": "text",
                        "analyzer": "ngram_analyzer"
                    },
                    "shingle": {
                        "type": "text",
                        "analyzer": "shingle_analyzer"
                    }
                }
            },
            "description": {"type": "text", "analyzer": "english"},
            "instructions": {"type": "text", "analyzer": "english"},
            "text": {"type": "text", "analyzer": "english"},
            "calories": {"type": "float"},
            "rating": {"type": "float"},
            "image_url": {"type": "keyword"}
        }
    }
}

es.indices.create(index=index_name, body=mapping)
print(f"Created index: {index_name}")

recipes_sample = recipes_df.head(sample_size)

def clean_text(text):
    """Remove unwanted characters like c("..."), quotes, and escape sequences."""
    if not isinstance(text, str):  
        return ""  

    text = re.sub(r'c\("', '', text)
    text = re.sub(r'"\)', '', text)

    text = text.replace('\\"', '')  
    text = text.replace('"', '')    
    text = text.replace("\\", '')  
    cleaned_urls = re.sub(r'\s+', ' ', text.strip())
    # Split the string by ', ' (comma followed by a space)
    urls = cleaned_urls.split(', ')
    return urls

def clean_instructions_combined_v2(instructions):
    """Cleans and formats recipe instructions into a single, well-structured sentence."""
    if isinstance(instructions, list):
        instructions = " ".join(instructions)  # Join list into a single string
    
    if not isinstance(instructions, str):
        return ""

    # Remove unwanted wrapping (c(...)) and extra quotes
    instructions = re.sub(r'c\s*\(\s*', '', instructions)  # Remove "c("
    instructions = re.sub(r'\s*\)$', '', instructions)  # Remove trailing ")"
    instructions = instructions.strip('"')

    # Fix letter-by-letter spacing issues (e.g., "M i x" → "Mix in")
    words = instructions.split()  # Split into words
    cleaned_words = []
    buffer = ""

    for word in words:
        if len(word) == 1:  # If it's a single letter, buffer it
            buffer += word
        else:
            if buffer:
                cleaned_words.append(buffer + word)  # Merge buffer with current word
                buffer = ""
            else:
                cleaned_words.append(word)

    if buffer:  # Append remaining buffered text
        cleaned_words.append(buffer)

    instructions = " ".join(cleaned_words)

    # Remove redundant punctuation and ensure proper spacing
    instructions = re.sub(r'\.\s*\.', '.', instructions)  # Remove repeated periods
    instructions = re.sub(r'\s*\.\s*', '. ', instructions)  # Ensure proper space after periods
    instructions = re.sub(r'\s*,', ',', instructions)  # Remove spaces before commas

    # Remove leading/trailing unwanted characters (like extra quotes)
    instructions = re.sub(r'(^\"|\"$)', '', instructions)  # Remove leading/trailing quotes

    # Ensure proper formatting of sentences and remove unnecessary escape sequences
    instructions = instructions.replace('\\"', '')  # Remove escaped quotes
    instructions = instructions.replace('", "', ', ')  # Convert improperly formatted list items into a natural sentence

    # Convert sentence breaks into commas for a continuous explanation
    instructions = re.sub(r'\s*\.\s*', ', ', instructions)  # Convert periods into commas for a smoother flow

    # Fix double commas and extra spaces
    instructions = re.sub(r',\s*,+', ', ', instructions)  # Remove repeated commas
    instructions = re.sub(r'\s+', ' ', instructions)  # Remove extra spaces

    # Ensure only one period at the very end
    instructions = instructions.strip().rstrip(',') + "."  # Remove trailing comma and add a period

    return instructions

def generate_docs(df):
    for idx, row in df.iterrows():
        recipe_id = str(int(float(row.get('RecipeId', idx))))
        name = row.get('Name', '')
        description = row.get('Description', '')  # Replacing "ingredients"
        instructions = clean_instructions_combined_v2(row.get('RecipeInstructions', []))  # ✅ Clean instructions at indexing
        text = clean_text(row.get('text', ''))  # Clean text
        calories = float(row.get('Calories', 0))
        image_url = row.get('image_link', [])

        try:
            rating = float(row['AggregatedRating']) if not np.isnan(row['AggregatedRating']) else 0
        except (KeyError, TypeError, ValueError):
            rating = 0
        
        doc = {
            "_op_type": "index",
            "_index": index_name,
            "_id": recipe_id,
            "_source": {
                "recipe_id": recipe_id,
                "name": name,
                "description": description,  # Updated field
                "instructions": instructions,  # ✅ Cleaned instructions
                "text": text,  # Updated field
                "calories": calories,
                "rating": rating,
                "image_url": image_url
            }
        }
        if not np.isnan(row.get('AggregatedRating', np.nan)):
            doc["_source"]["rating"] = float(row["AggregatedRating"])
        yield doc


bulk(es, generate_docs(recipes_sample))

print(f"Indexed {len(recipes_sample)} recipes into Elasticsearch.")


C:\Users\Ned\AppData\Local\Temp\ipykernel_74668\1154701266.py:11: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=index_name, ignore=[400, 404])


Created index: recipes
Indexed 1000 recipes into Elasticsearch.


### Flask

In [ ]:
import uuid
import pymysql
import numpy as np
from flask import Flask, request, jsonify
from flask_cors import CORS
from werkzeug.security import generate_password_hash, check_password_hash
from elasticsearch import Elasticsearch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
app = Flask(__name__)
app.secret_key = "SOME_RANDOM_SECRET_KEY"
CORS(app)

def get_db_connection():
    return pymysql.connect(
        host='127.0.0.1',
        port=3309,
        user='root',
        password='root_password',
        db='my_database',
        cursorclass=pymysql.cursors.DictCursor
    )


INDEX_NAME = "recipes"

@app.route('/search', methods=['GET'])
def search():
    query = request.args.get('q', '').strip()
    page = int(request.args.get('page', 1))
    results_per_page = 10

    if not query:
        return jsonify({"error": "Query parameter 'q' is required"}), 400

    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            cursor.execute("INSERT INTO search_logs (query) VALUES (%s)", (query,))
            connection.commit()
    except Exception as log_error:
        print("Search log error:", log_error)
    finally:
        connection.close()

    es_query = {
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["name^3", "description^2", "text", "name.ngram^2"],
                "fuzziness": "AUTO"
            }
        },
        "size": results_per_page,
        "from": (page - 1) * results_per_page,
        "suggest": {
            "text": query,
            "phrase_suggest": {
                "phrase": {
                    "field": "name.shingle",
                    "size": 1,
                    "gram_size": 3,
                    "direct_generator": [{"field": "name.shingle", "suggest_mode": "always"}],
                    "highlight": {"pre_tag": "<em>", "post_tag": "</em>"}
                }
            }
        }
    }

    try:
        response = es.search(index=INDEX_NAME, body=es_query)
        total_count = response["hits"]["total"]["value"]
        hits = response.get("hits", {}).get("hits", [])

        suggested_query = ''
        suggestions = response.get("suggest", {}).get("phrase_suggest", [])
        if suggestions and suggestions[0]["options"]:
            suggested_query = suggestions[0]["options"][0]["text"]

        results = [{
            "recipe_id": hit["_source"].get("recipe_id"),
            "name": hit["_source"].get("name"),
            "description": hit["_source"].get("description"),
            "instructions": hit["_source"].get("instructions"),
            "image_url": clean_text(hit["_source"].get("image_url")),
            "calories": hit["_source"].get("calories"),
            "rating": hit["_source"].get("rating", 0),
            "score": hit["_score"]
        } for hit in hits]

        return jsonify({
            "results": results,
            "total_count": total_count,
            "suggested_query": suggested_query
        })

    except Exception as e:
        return jsonify({"error": f"Elasticsearch error: {str(e)}"}), 500

# ------------------ Recipe Details ------------------ #
@app.route('/recipe/<recipe_id>', methods=['GET'])
def get_recipe(recipe_id):
    response = es.get(index=INDEX_NAME, id=recipe_id, ignore=[404])
    if not response or not response.get("found"):
        return jsonify({"error": f"Recipe ID {recipe_id} not found"}), 404

    source = response["_source"]
    source["instructions"] = clean_instructions_combined_v2(source.get("instructions", ""))
    
    cleaned_image = clean_text(source.get("image_url", ""))
    source["image_url"] = cleaned_image[0] if isinstance(cleaned_image, list) and cleaned_image else ""

    return jsonify(source)


# ------------------ User Registration/Login ------------------ #
@app.route('/register', methods=['POST'])
def register():
    data = request.json
    username, password = data.get('username'), data.get('password')
    if not username or not password:
        return jsonify({"error": "Missing username or password"}), 400

    password_hash = generate_password_hash(password)
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT user_id FROM users WHERE username = %s", (username,))
            if cursor.fetchone():
                return jsonify({"error": "Username already taken"}), 400
            cursor.execute("INSERT INTO users (username, password_hash) VALUES (%s, %s)", (username, password_hash))
            connection.commit()
        return jsonify({"message": "Registration successful"}), 200
    finally:
        connection.close()

@app.route('/login', methods=['POST'])
def login():
    data = request.json
    username, password = data.get('username'), data.get('password')
    if not username or not password:
        return jsonify({"error": "Missing username or password"}), 400

    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT user_id, password_hash FROM users WHERE username = %s", (username,))
            user = cursor.fetchone()
            if not user or not check_password_hash(user['password_hash'], password):
                return jsonify({"error": "Invalid username or password"}), 401

            token = str(uuid.uuid4())
            cursor.execute("REPLACE INTO sessions (token, username, user_id) VALUES (%s, %s, %s)",
                           (token, username, user['user_id']))
            connection.commit()
            return jsonify({"message": "Login successful", "username": username, "token": token}), 200
    finally:
        connection.close()

@app.route('/logout', methods=['POST'])
def logout():
    auth_header = request.headers.get('Authorization')
    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401
    token = auth_header.split(" ")[1]

    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            cursor.execute("DELETE FROM sessions WHERE token = %s", (token,))
            connection.commit()
        return jsonify({"message": "Logged out successfully"}), 200
    finally:
        connection.close()

# ------------------ Top Searches ------------------ #
@app.route('/top-searches', methods=['GET'])
def top_searches():
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            cursor.execute("""
                SELECT query, COUNT(*) AS count 
                FROM search_logs 
                GROUP BY query 
                ORDER BY count DESC 
                LIMIT 5
            """)
            top_queries = cursor.fetchall()
        return jsonify({"top_searches": top_queries}), 200
    except Exception as e:
        return jsonify({"error": f"Failed to fetch top searches: {str(e)}"}), 500
    finally:
        connection.close()

# ------------------ Recommendations ------------------ #
@app.route('/recommendations', methods=['GET'])
def recommendations():
    auth_header = request.headers.get('Authorization')
    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401
    token = auth_header.split(" ")[1]

    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT user_id FROM sessions WHERE token = %s", (token,))
            user = cursor.fetchone()
            if not user:
                return jsonify({"error": "Invalid token"}), 401
            user_id = user['user_id']

            cursor.execute("SELECT recipe_id FROM bookmarks WHERE user_id = %s", (user_id,))
            bookmarked_ids = [r["recipe_id"] for r in cursor.fetchall()]

        bookmarked_recipes = []
        if bookmarked_ids:
            import random
            random.shuffle(bookmarked_ids)
            picked = bookmarked_ids[:2]
            for rid in picked:
                res = es.get(index=INDEX_NAME, id=str(rid), ignore=[404])
                if res and res.get("found"):
                    doc = res["_source"]
                    doc["image_url"] = clean_text(doc.get("image_url", ""))
                    bookmarked_recipes.append(doc)

        must_not_clause = []
        if bookmarked_ids:
            must_not_clause.append({"terms": {"recipe_id": [str(i) for i in bookmarked_ids]}})

        es_query = {
            "query": {
                "bool": {
                    "must": [{"exists": {"field": "image_url"}}],
                    "must_not": must_not_clause
                }
            },
            "size": 3
        }

        es_res = es.search(index=INDEX_NAME, body=es_query)
        new_recipes = []
        for hit in es_res.get("hits", {}).get("hits", []):
            doc = hit["_source"]
            doc["image_url"] = clean_text(doc.get("image_url", ""))
            new_recipes.append(doc)

        return jsonify({"recommendations": bookmarked_recipes + new_recipes}), 200
    except Exception as e:
        return jsonify({"error": f"Error fetching recommendations: {str(e)}"}), 500
    finally:
        connection.close()

@app.route('/bookmark', methods=['POST'])
def add_bookmark():
    data = request.json or {}
    auth_header = request.headers.get('Authorization')
    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401
    token = auth_header.split(" ")[1]

    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT username FROM sessions WHERE token = %s", (token,))
            user = cursor.fetchone()
            if not user:
                return jsonify({"error": "Invalid token"}), 401
            username = user['username']

            recipe_id = data.get('recipe_id')
            folder_name = data.get('folder_name', '').strip()
            rating = data.get('rating', 0)

            if not recipe_id or not folder_name:
                return jsonify({"error": "Missing recipe_id or folder_name"}), 400

            cursor.execute("SELECT id FROM folders WHERE username = %s AND folder_name = %s", (username, folder_name))
            folder_row = cursor.fetchone()
            folder_id = folder_row['id'] if folder_row else None

            if not folder_id:
                cursor.execute("INSERT INTO folders (username, folder_name) VALUES (%s, %s)", (username, folder_name))
                folder_id = cursor.lastrowid

            cursor.execute("INSERT IGNORE INTO folder_recipes (folder_id, recipe_id) VALUES (%s, %s)", (folder_id, recipe_id))
            cursor.execute("""
                INSERT INTO bookmarks (user_id, recipe_id, rating)
                VALUES ((SELECT user_id FROM users WHERE username = %s), %s, %s)
                ON DUPLICATE KEY UPDATE rating = VALUES(rating)
            """, (username, recipe_id, rating))
            connection.commit()

        return jsonify({"message": "Bookmark added successfully"}), 200
    except Exception as e:
        return jsonify({"error": f"Bookmark error: {str(e)}"}), 500
    finally:
        connection.close()

@app.route('/bookmarks', methods=['GET'])
def get_bookmarks():
    auth_header = request.headers.get('Authorization')
    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401
    token = auth_header.split(" ")[1]

    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT username FROM sessions WHERE token = %s", (token,))
            user = cursor.fetchone()
            if not user:
                return jsonify({"error": "Invalid token"}), 401
            username = user['username']

            cursor.execute("SELECT id AS folder_id, folder_name FROM folders WHERE username = %s", (username,))
            folders = cursor.fetchall()

            for folder in folders:
                folder_id = folder["folder_id"]
                cursor.execute("""
                    SELECT fr.recipe_id, b.rating 
                    FROM folder_recipes fr
                    LEFT JOIN bookmarks b ON fr.recipe_id = b.recipe_id 
                    AND b.user_id = (SELECT user_id FROM users WHERE username = %s)
                    WHERE fr.folder_id = %s
                    ORDER BY b.rating DESC
                """, (username, folder_id))
                recipes = cursor.fetchall()

                for recipe in recipes:
                    es_res = es.get(index="recipes", id=str(recipe["recipe_id"]), ignore=[404])
                    if es_res and es_res.get("found"):
                        src = es_res["_source"]
                        recipe["image_url"] = clean_text(src.get("image_url", ""))
                        recipe["name"] = src.get("name", "")
                        recipe["description"] = src.get("description", "")

                folder["recipes"] = recipes

        return jsonify({"folders": folders}), 200
    except Exception as e:
        return jsonify({"error": f"Bookmarks fetch error: {str(e)}"}), 500
    finally:
        connection.close()

@app.route('/bookmark', methods=['DELETE'])
def delete_bookmark():
    data = request.json or {}
    auth_header = request.headers.get('Authorization')
    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401
    token = auth_header.split(" ")[1]

    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT username FROM sessions WHERE token = %s", (token,))
            user = cursor.fetchone()
            if not user:
                return jsonify({"error": "Invalid token"}), 401
            username = user['username']

            folder_id = data.get('folder_id')
            recipe_id = data.get('recipe_id')
            if not folder_id or not recipe_id:
                return jsonify({"error": "Missing folder_id or recipe_id"}), 400

            cursor.execute("DELETE FROM folder_recipes WHERE folder_id = %s AND recipe_id = %s", (folder_id, recipe_id))
            cursor.execute("""
                DELETE FROM bookmarks 
                WHERE recipe_id = %s AND user_id = 
                (SELECT user_id FROM users WHERE username = %s)
                AND NOT EXISTS (SELECT 1 FROM folder_recipes WHERE recipe_id = %s)
            """, (recipe_id, username, recipe_id))
            connection.commit()

        return jsonify({"message": "Bookmark deleted successfully"}), 200
    except Exception as e:
        return jsonify({"error": f"Delete bookmark error: {str(e)}"}), 500
    finally:
        connection.close()

@app.route('/folder', methods=['DELETE'])
def delete_folder():
    data = request.json or {}
    auth_header = request.headers.get('Authorization')
    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401
    token = auth_header.split(" ")[1]

    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT username FROM sessions WHERE token = %s", (token,))
            user = cursor.fetchone()
            if not user:
                return jsonify({"error": "Invalid token"}), 401
            username = user['username']

            folder_id = data.get('folder_id')
            if not folder_id:
                return jsonify({"error": "Missing folder_id"}), 400

            cursor.execute("DELETE FROM folder_recipes WHERE folder_id = %s", (folder_id,))
            cursor.execute("DELETE FROM folders WHERE id = %s AND username = %s", (folder_id, username))
            connection.commit()

        return jsonify({"message": "Folder and bookmarks deleted successfully"}), 200
    except Exception as e:
        return jsonify({"error": f"Delete folder error: {str(e)}"}), 500
    finally:
        connection.close()

@app.route('/suggestions/<int:folder_id>', methods=['GET'])
def generate_suggestions(folder_id):
    auth_header = request.headers.get('Authorization')
    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401
    token = auth_header.split(" ")[1]

    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            cursor.execute("SELECT user_id FROM sessions WHERE token = %s", (token,))
            user = cursor.fetchone()
            if not user:
                return jsonify({"error": "Invalid token"}), 401
            user_id = user['user_id']

            cursor.execute("SELECT recipe_id FROM folder_recipes WHERE folder_id = %s", (folder_id,))
            folder_recipes = cursor.fetchall()

        if not folder_recipes:
            return jsonify({"error": "Folder is empty, no suggestions can be generated."}), 400

        folder_recipe_ids = [str(r['recipe_id']) for r in folder_recipes]

        folder_texts = []
        for rid in folder_recipe_ids:
            res = es.get(index=INDEX_NAME, id=rid, ignore=[404])
            if res.get("found"):
                src = res["_source"]
                combined = f"{src.get('name', '')} {src.get('description', '')} {src.get('instructions', '')}"
                folder_texts.append(combined.strip())

        es_query = {
            "query": {
                "bool": {
                    "must_not": {
                        "terms": {"recipe_id": folder_recipe_ids}
                    }
                }
            },
            "size": 100
        }

        es_res = es.search(index=INDEX_NAME, body=es_query)
        candidates = es_res.get("hits", {}).get("hits", [])

        candidate_texts = []
        candidate_meta = []
        for hit in candidates:
            src = hit["_source"]
            combined = f"{src.get('name', '')} {src.get('description', '')} {src.get('instructions', '')}"
            candidate_texts.append(combined.strip())
            candidate_meta.append({
                "recipe_id": src.get("recipe_id"),
                "name": src.get("name"),
                "description": src.get("description"),
                "instructions": src.get("instructions"),
                "image_url": clean_text(src.get("image_url", "")),
                "calories": src.get("calories"),
                "rating": src.get("rating", 0),
                "score": 0
            })

        all_texts = folder_texts + candidate_texts
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform(all_texts)

        folder_vecs = tfidf_matrix[:len(folder_texts)]
        candidate_vecs = tfidf_matrix[len(folder_texts):]

        sim_matrix = cosine_similarity(candidate_vecs, folder_vecs)
        sim_scores = np.mean(sim_matrix, axis=1)

        for i, score in enumerate(sim_scores):
            candidate_meta[i]['score'] = float(score)

        ranked = sorted(candidate_meta, key=lambda x: x['score'], reverse=True)[:10]
        return jsonify({"suggestions": ranked}), 200
    except Exception as e:
        return jsonify({"error": f"Suggestion error: {str(e)}"}), 500
    finally:
        connection.close()


In [25]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
C:\Users\Ned\AppData\Local\Temp\ipykernel_74668\3721899985.py:94: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es.get(index=INDEX_NAME, id=recipe_id, ignore=[404])
127.0.0.1 - - [16/Mar/2025 18:58:40] "GET /recipe/38 HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 18:58:41] "OPTIONS /recommendations HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 18:58:41] "GET /top-searches HTTP/1.1" 200 -
C:\Users\Ned\AppData\Local\Temp\ipykernel_74668\3721899985.py:213: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  res = es.get(index=INDEX_NAME, id=str(rid), ignore=[404])
127.0.0.1 - - [16/Mar/2025 18:58:41] "GET /recommendations HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 18:58:42] "GET /recipe/538 HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 18:58:42] "GET /top-searches HTTP/1.1" 200 -
127.0.0.1 -